---
title: "Some tests with gediDB"
execute: 
  enabled: true
---

In [ ]:
import geopandas as gpd
import gedidb as gdb
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio
import numpy as np
import os

# Instantiate the GEDIProvider
provider = gdb.GEDIProvider(
    storage_type='s3',
    s3_bucket="dog.gedidb.gedi-l2-l4-v002",
    url="https://s3.gfz-potsdam.de"
)

# Load region of interest (ROI)
region_of_interest = gpd.read_file('MougueirasFire.geojson')

# Define variables to query and quality filters
vars_selected = ["agbd", 'wsci']

# Query data
gedi_data = provider.get_data(
    variables=vars_selected,
    query_type="bounding_box",
    geometry=region_of_interest,
    start_time="2019-09-26",
    end_time="2020-09-27",
    return_type='dataframe'
)

gdf = gpd.GeoDataFrame(gedi_data, geometry=gpd.points_from_xy(x= gedi_data['longitude'], y = gedi_data['latitude']))
gdf.crs = 4326

gedi_ba = gpd.sjoin(gdf, region_of_interest, how='inner')

In [ ]:
#| title: GDP and Life Expectancy
pio.renderers.default = "iframe"

mb_token = os.environ.get('MAPBOX_TOKEN')

px.set_mapbox_access_token(mb_token)

fig = ff.create_hexbin_mapbox(
    data_frame=gedi_ba, lat="latitude", 
    lon="longitude", color = 'agbd',
    color_continuous_scale = 'Greens', agg_func=np.mean,
    nx_hexagon=15, opacity=0.9, labels={"color": "AGBD [Mg/ha]"},
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()